# **ETL RETAIL DATA**

## **1. Extraction**

___
### *Imports and Setting Seeds*

In [1]:
import pandas as pd 
import numpy as np
import random
from datetime import datetime, timedelta
from faker import Faker

# ===== Reproducibility =====
# Fix the randomness so the output is the same every time  the script runs
np.random.seed(42)
random.seed(42)

#### *Explanation:*

Import needed libraries. Set seeds so random functions produce the same results every run (important for debugging and reproducibility).
___

### *Configuration / Constants*

In [2]:
# ===== Config =====
NUM_ROWS = 1000           # Number of rows (records) to generate
NUM_CUSTOMERS = 100       # Number of unique customers
START_DATE = datetime(2023, 8, 12)   # Start date for invoices
END_DATE = datetime(2025, 8, 11)     # End date for invoices

# Countries for customers
COUNTRIES = ["United Kingdom", "Germany", "France", "Spain", "Netherlands", "Italy", "Norway", "Portugal"]

# List of products (StockCode, Description, Category)
PRODUCTS = [
    ("E001", "Wireless Mouse", "Electronics"),
    ("E002", "Bluetooth Headphones", "Electronics"),
    ("E003", "Smartphone Charger", "Electronics"),
    ("C001", "Men's T-Shirt", "Clothing"),
    ("C002", "Women's Jeans", "Clothing"),
    ("C003", "Baseball Cap", "Clothing"),
    ("H001", "Ceramic Mug", "Home Goods"),
    ("H002", "Wall Clock", "Home Goods"),
    ("H003", "LED Desk Lamp", "Home Goods"),
    ("T001", "Stuffed Bear", "Toys"),
    ("T002", "Building Blocks Set", "Toys"),
    ("T003", "RC Car", "Toys"),
]


#### *Explanation:*

Define constants such as the number of rows, number of customers, date range, list of countries, and product details to be used throughout the data generation process.

___


### *Function to Generate Random Invoice Dates*

In [3]:
def random_date(start_date, end_date):
    """Generate a random datetime between start and end dates."""
    delta = end_date - start_date
    return start_date + timedelta(
        days=random.randint(0, delta.days),
        seconds=random.randint(0, 86400)  # seconds in a day
    )


#### *Explanation:*

Create a helper function to generate random dates between the specified start and end dates for invoice timestamps.

___

### *Generate Base Synthetic Dataset*

In [4]:
def generate_base_data(num_rows):
    """Generate synthetic retail data with given number of rows."""
    data = []
    for _ in range(num_rows):
        invoice_no = f"INV{random.randint(10000, 99999)}"   # Random invoice number
        stock_code, description, category = random.choice(PRODUCTS)  # Random product info
        quantity = random.randint(1, 50)                      # Quantity between 1 and 50
        invoice_date = random_date(START_DATE, END_DATE)     # Random invoice date
        unit_price = round(random.uniform(1, 100), 2)        # Unit price between 1 and 100
        customer_id = f"CUST{random.randint(1, NUM_CUSTOMERS)}"  # Random customer ID
        country = random.choice(COUNTRIES)                    # Random country
        data.append([invoice_no, stock_code, description, category, quantity, invoice_date, unit_price, customer_id, country])
    
    columns = ["InvoiceNo", "StockCode", "Description", "Category", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country"]
    return pd.DataFrame(data, columns=columns)


#### *Explanation:*

Generate the main synthetic dataset with random invoices, product info, quantities, invoice dates, prices, customer IDs, and countries.

___

### *Inject Missing Values*

In [5]:
def inject_missing_values(df, desc_frac=0.02, country_frac=0.01):
    """Randomly replace some 'Description' and 'Country' values with NaN."""
    df.loc[df.sample(frac=desc_frac).index, "Description"] = np.nan
    df.loc[df.sample(frac=country_frac).index, "Country"] = np.nan
    return df


#### *Explanation:*

Simulate data imperfections by injecting missing values randomly into the 'Description' and 'Country' columns to mimic real-world dirty data.

___

### *Inject Outliers*

In [6]:
def inject_outliers(df, neg_qty_frac=0.01, zero_price_frac=0.01):
    """Inject outliers: negative quantity and zero prices."""
    df.loc[df.sample(frac=neg_qty_frac).index, "Quantity"] *= -1  # Negative quantity (returns)
    df.loc[df.sample(frac=zero_price_frac).index, "UnitPrice"] = 0  # Zero price (data error)
    return df


#### *Explanation:*

Introduce data anomalies by randomly making some quantities negative and unit prices zero, representing common outlier issues.

___

### *Save DataFrame to CSV*

In [7]:
def save_dataset(df, filename):
    """Save DataFrame to a CSV file."""
    df.to_csv(filename, index=False)


#### *Explanation:*

Save the generated DataFrame to a CSV file for persistence and later use.

___

### *Generate Customer Names with Faker*

In [8]:
def generate_customer_names(num_customers):
    """Generate fake customer names mapped by CustomerID."""
    fake = Faker()
    return {f"CUST{i}": fake.name() for i in range(1, num_customers + 1)}


#### *Explanation:*

Use the Faker library to generate realistic and unique customer names, mapped one-to-one with customer IDs.

___

### *Assign Customer Names to DataFrame*

In [9]:
def assign_customer_names(df, customer_names):
    """Add 'CustomerName' column by mapping CustomerID."""
    df['CustomerName'] = df['CustomerID'].map(customer_names)
    return df


#### *Explanation:*

Add a new column 'CustomerName' to the DataFrame by mapping existing customer IDs to the generated customer names.

___

### *Pipeline Execution (Putting It All Together)*

In [10]:
df = generate_base_data(NUM_ROWS)        # Step 1: Generate base data
df = inject_missing_values(df)           # Step 2: Add missing values
df = inject_outliers(df)                  # Step 3: Add outliers

# Convert InvoiceDate to string for saving to CSV
df["InvoiceDate"] = df["InvoiceDate"].astype(str)

NUM_CUSTOMERS = df['CustomerID'].nunique()  # Count unique customers

customer_names = generate_customer_names(NUM_CUSTOMERS)  # Generate names
df = assign_customer_names(df, customer_names)           # Assign names to df

save_dataset(df, "synthetic_retail_dataset.csv")         # Save as CSV

print(df.head())  # Show first few rows as a check


  InvoiceNo StockCode           Description     Category  Quantity  \
0  INV93810      E002  Bluetooth Headphones  Electronics         2   
1  INV98696      T003                RC Car         Toys        35   
2  INV38657      C001         Men's T-Shirt     Clothing        33   
3  INV38893      H002            Wall Clock   Home Goods        38   
4  INV54597      C002         Women's Jeans     Clothing        10   

           InvoiceDate  UnitPrice CustomerID  Country    CustomerName  
0  2024-05-19 08:54:58      23.10     CUST95  Germany  Nathaniel Ross  
1  2023-11-09 21:29:57      42.77      CUST4  Germany   Craig Perkins  
2  2025-04-19 00:57:58      56.56     CUST92   Norway    Ruben Murphy  
3  2024-05-22 00:14:11      76.12     CUST21   Norway    Lance Harris  
4  2024-03-19 12:15:18      11.12     CUST49  Germany    Stacie Allen  


#### *Explanation:*

Run all functions in sequence to generate, clean, enrich, and save the synthetic retail dataset, then print the first few rows for verification.
___

### *Data Inspection Summary*

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   InvoiceNo     1000 non-null   object 
 1   StockCode     1000 non-null   object 
 2   Description   980 non-null    object 
 3   Category      1000 non-null   object 
 4   Quantity      1000 non-null   int64  
 5   InvoiceDate   1000 non-null   object 
 6   UnitPrice     1000 non-null   float64
 7   CustomerID    1000 non-null   object 
 8   Country       990 non-null    object 
 9   CustomerName  1000 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 78.3+ KB


#### *Explanation:*

- Dataset has 1000 rows and 10 columns.
- Columns include IDs, product info, transaction details, and customer info.
- Data types: mostly strings (object), Quantity (int), UnitPrice (float), InvoiceDate as string.
- Missing values: Description (2%), Country (1%).
- InvoiceDate should be converted to datetime for analysis.


___
### *Handle Missing Values and Convert InvoiceDate to Datetime - the generated data*

In [12]:
def clean_and_convert(df):

    #Handle missing values and convert InvoiceDate to datetime.

    # Fill missing values
    df['Description'] = df['Description'].fillna('Unknown Product')
    df['Country'] = df['Country'].fillna('Unknown Country')

    # Convert InvoiceDate to datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

    return df

# Usage
df = clean_and_convert(df)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   InvoiceNo     1000 non-null   object        
 1   StockCode     1000 non-null   object        
 2   Description   1000 non-null   object        
 3   Category      1000 non-null   object        
 4   Quantity      1000 non-null   int64         
 5   InvoiceDate   1000 non-null   datetime64[ns]
 6   UnitPrice     1000 non-null   float64       
 7   CustomerID    1000 non-null   object        
 8   Country       1000 non-null   object        
 9   CustomerName  1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 78.3+ KB
None


#### *Explanation:*

- Fill missing values in the **Description** column with the string `'Unknown Product'` to avoid nulls that might break analysis or database constraints.
- Fill missing values in the **Country** column with `'Unknown Country'` for the same reason.
- Convert the **InvoiceDate** column from string to pandas datetime format to enable time-based operations.
- Use `errors='coerce'` to convert invalid date strings into `NaT` (Not a Time), which represents missing datetime values.

___


## **2. Transformation**

### *Calculate TotalSales Column*

In [13]:
def add_total_sales(df):
    df['TotalSales'] = df['Quantity'] * df['UnitPrice']
    return df


#### *Explanation:*

Create a new column `TotalSales` by multiplying `Quantity` and `UnitPrice` for each record. This provides the sales revenue per transaction.

___

### *Remove Outliers*

In [14]:
def remove_outliers(df):
    cleaned_df = df[(df['Quantity'] >= 0) & (df['UnitPrice'] > 0)]
    return cleaned_df


#### *Explanation:*

Filter out rows where `Quantity` is negative or `UnitPrice` is zero or negative to ensure data accuracy and quality.

___

### *Filter Sales in the Last Year*

In [15]:
from datetime import datetime

def filter_last_year_sales(df, reference_date=datetime(2025, 8, 12)):
    one_year_ago = reference_date.replace(year=reference_date.year - 1)
    filtered_df = df[df['InvoiceDate'] >= one_year_ago]
    return filtered_df


#### *Explanation:*

Keep only transactions within the last year, using August 12, 2025, as the reference date. This focuses analysis on recent sales activity.

___

### *Create Customer Summary*

In [16]:
def create_customer_summary(df):
    customer_summary = df.groupby('CustomerID').agg({
        'TotalSales': 'sum',
        'Country': 'first'  
    }).reset_index()
    customer_summary.rename(columns={'TotalSales': 'TotalPurchases'}, inplace=True)
    return customer_summary


#### *Explanation:*

Group data by `CustomerID` to create a summary table that aggregates total purchases per customer and retains their country information.

___

### *Transform Pipeline*

In [17]:
def transform_pipeline(df):
    df = add_total_sales(df)
    df = remove_outliers(df)
    df = filter_last_year_sales(df)
    customer_summary = create_customer_summary(df)
    return df, customer_summary


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   InvoiceNo     1000 non-null   object        
 1   StockCode     1000 non-null   object        
 2   Description   1000 non-null   object        
 3   Category      1000 non-null   object        
 4   Quantity      1000 non-null   int64         
 5   InvoiceDate   1000 non-null   datetime64[ns]
 6   UnitPrice     1000 non-null   float64       
 7   CustomerID    1000 non-null   object        
 8   Country       1000 non-null   object        
 9   CustomerName  1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 78.3+ KB


#### *Explanation:*

Run all transformation functions sequentially to produce cleaned, enriched sales data and a customer summary dimension table, ready for loading or further analysis.
___

## **3. Load**

In [19]:
import sqlite3
import pandas as pd


# Connect to (or create) the SQLite database
conn = sqlite3.connect('retail_dw.db')
cursor = conn.cursor()

# Step 1: Create tables using your saved .sql script
with open('retail_dw.sql', 'r') as f:
    sql_script = f.read()
cursor.executescript(sql_script)
print("Tables created successfully!")

# Step 2: Load data into dimension tables
#  for dim_date
df_date = df[['InvoiceDate']].drop_duplicates().copy()
df_date['full_date'] = df_date['InvoiceDate']
df_date['day'] = df_date['InvoiceDate'].dt.day
df_date['month'] = df_date['InvoiceDate'].dt.month
df_date['quarter'] = df_date['InvoiceDate'].dt.quarter
df_date['year'] = df_date['InvoiceDate'].dt.year
df_date.reset_index(drop=True, inplace=True)
df_date['date_id'] = df_date.index + 1  # create PK

# Then insert into SQLite
df_date_to_insert = df_date[['date_id', 'full_date', 'day', 'month', 'quarter', 'year']]

# for dim_customer
# Prepare dim_customer DataFrame to match SQLite table
df_customer = df[['CustomerID','Country']].drop_duplicates().copy()
df_customer['customer_id'] = range(1, len(df_customer)+1)
df_customer = df_customer.rename(columns={
    'CustomerID': 'customer_code',   # match the table column
    'Country': 'country'
})
# Reorder columns to match the table
df_customer = df_customer[['customer_id','customer_code','country']]
df_customer.to_sql('dim_customer', conn, if_exists='append', index=False)

#  for dim_product
# Prepare dim_product DataFrame to match SQLite table
df_product = df[['StockCode','Description','Category']].drop_duplicates().copy()
df_product['product_id'] = range(1, len(df_product)+1)

# Rename columns to match SQLite table
df_product = df_product.rename(columns={
    'StockCode': 'stock_code',
    'Description': 'name',
    'Category': 'category'
})

# Reorder columns to match the table
df_product = df_product[['product_id','stock_code','name','category']]

# Load into SQLite
df_product.to_sql('dim_product', conn, if_exists='append', index=False)


# Step 3: Load data into fact table
df_fact = df.copy()

# Merge with date dimension
df_fact = df_fact.merge(df_date[['date_id', 'InvoiceDate']], 
                        left_on='InvoiceDate', right_on='InvoiceDate', how='left')

# Merge with customer dimension
df_fact = df_fact.merge(df_customer[['customer_id', 'customer_code']], 
                        left_on='CustomerID', right_on='customer_code', how='left')

# Merge with product dimension
df_fact = df_fact.merge(df_product[['product_id', 'stock_code']], 
                        left_on='StockCode', right_on='stock_code', how='left')

# Calculate TotalSales
df_fact['TotalSales'] = df_fact['Quantity'] * df_fact['UnitPrice']

## Select & rename columns to match fact_sales schema
df_fact_final = df_fact[['InvoiceNo','date_id','product_id','customer_id','Quantity','UnitPrice','TotalSales']]
df_fact_final = df_fact_final.rename(columns={
    'InvoiceNo': 'invoice_no',
    'Quantity': 'quantity',
    'UnitPrice': 'unit_price',
    'TotalSales': 'total_sales'
})
# Load into SQLite
df_fact_final.to_sql('fact_sales', conn, if_exists='append', index=False)

conn.commit()
conn.close()
print("Data loaded successfully into SQLite!")

Tables created successfully!


Data loaded successfully into SQLite!
